# Setting up a Graph Database for the US Government IT Portfolios

In [1]:
import pandas as pd
import numpy as np
import dabl
%matplotlib inline
import os
from dabl import plot
import matplotlib.pyplot as plt
import datetime as dt

Matplotlib is building the font cache; this may take a moment.


In [2]:
directory = '/Users/lawrence/Documents/GitHub/data_files_for_access/ProjectHack3data_Challenge9_from_Said_school/2013/'

# Bring in Projects file

In [3]:
Projects=pd.read_csv(directory+'Projects.csv', encoding= 'unicode_escape')

In [4]:
Projects=Projects.drop(['Agency Project ID','Project Description','Lifecycle Cost', 'Cost Variance (%)','Projected/Actual Cost ($ M)','Updated Time','Unique Project ID'],axis=1)

In [5]:
Projects['Completion Date (B1)']=Projects['Completion Date (B1)'].astype('datetime64')
Projects['Planned Project Completion Date (B2)']=Projects['Planned Project Completion Date (B2)'].astype('datetime64')
Projects['Start Date']=Projects['Start Date'].astype('datetime64')
Projects['Projected/Actual Project Completion Date (B2)']=Projects['Projected/Actual Project Completion Date (B2)'].astype('datetime64')
Projects['Updated Date']=Projects['Updated Date'].astype('datetime64')
Projects['Business Case ID']=Projects['Business Case ID'].astype('category')
Projects['Agency Code']=Projects['Agency Code'].astype('category')
Projects['Project ID']=Projects['Project ID'].astype('category')

In nearly all cases, Completion Date (B1) > Planned Project Completion Date (B2)

In some cases 'Projected/Actual Project Completion Date (B2) < Planned Project Completion Date (B2)
But in most cases, it is the other way around , so that Planned < Project /Actual i,e is earlier

And in most cases, where not equal: Projected/Actual Project Completion Date (B2)' <   'Completion Date (B1)'

therefore, Planning progresses in this order 

1. Planned Project Completion Date (B2)
2. Projected/Actual Project Completion Date (B2)
3. Completion Date (B1) 

So I will create:

- Planned Duration
 - Project Delay, which is the worst Project Delay I can find
Schedule variance appears to be Planned Project Completion Date (B2) MINUS 'Projected/Actual Project Completion Date (B2)
and Hence is often listed as Negative.
Which is okay, so Negative just means that Progress was underwhelming as normal.
So I can drop schedule variance and schedule  variance % because we have calculated it better as Project_Delay

In [6]:
# worked this out by a variety of boolean mixing such as:
# constraint=(Projects['Projected/Actual Project Completion Date (B2)']<Projects['Completion Date (B1)'])&(pd.notna(Projects['Projected/Actual Project Completion Date (B2)']))&(pd.notna(Projects['Completion Date (B1)']))
# Projects [constraint]

Projected/Actual more or less the same as Planned cost.
- sometimes a little more which makes sense
- a few with 0s which is okay
- so maybe best to drop 'Projected/Actual'

Lifecycle cost is:
- often zero
- mostly identical to Planned
- so best to drop this too. 
Looks like mostly Cost Variance +Planned Cost = Project/Actual Cost
i.e. just keep Planned cost and Cost Variance

In [7]:
# worked this out from variants of: 
# plt.scatter ((Projects['Planned Cost ($ M)'],Projects['Cost Variance ($ M)']))

In [6]:
Projects['Planned_Duration']=(Projects['Planned Project Completion Date (B2)']-Projects['Start Date']).dt.days
Projects['Project_delay']=(np.maximum(Projects['Projected/Actual Project Completion Date (B2)']-Projects['Planned Project Completion Date (B2)'],Projects['Completion Date (B1)']-Projects['Planned Project Completion Date (B2)'])).dt.days

In [7]:
Projects=Projects.drop(['Planned Project Completion Date (B2)','Completion Date (B1)','Projected/Actual Project Completion Date (B2)'],axis=1)

In [8]:
Projects['Start_days_after_2000']=(Projects['Start Date']-pd.Timestamp('2000-01-01')).dt.days

In [9]:
Projects=Projects.drop(['Schedule Variance (%)','Schedule Variance (in days)'],axis=1)

In [10]:
Projects['Updated Date']=(Projects['Updated Date']-Projects['Start Date']).dt.days # where this is now the no of days the update happened after start date

In [33]:
Constraint=Projects['Agency Code']==9

In [35]:
Projects=Projects[Constraint]

## Work out Data Structure as you go. 
Agency >> Investment (w Business Case ) >> Project ( Start, Completion, Cost, Variances)

## Create a basic example in Neo4j inline

CREATE (m:Agency {name:'Dept Agriculture',Code:5})
CREATE (n:Investment {name:'AMS Infrastructure WAN and DMZ (AMSWAN)', Identifier:'005-000001723'})
CREATE (o:Project {name:'Virtualization', Id:657,Cost_Variance:0,Planned_Cost:0.179,Updated_Date:60,Planned_Duration:182, Project_delay:0,Start_days_after_2000:4291})
CREATE (m)-[:invests]->(n)
CREATE (n)-[:pays_for]->(o)

After testing this, I can bring in CSV: see further below

## Export to CSV en route to Neo4j

In [36]:
Projects=Projects.rename(columns={"Agency Name": "Agency_Name", "Agency Code":"Agency_Code",'Investment Title':'Investment_Title','Unique Investment Identifier':'Unique_Investment_Identifier','Project Name':'Project_Name','Project ID':'Project_ID','Start Date':'Start_Date', 'Cost Variance ($ M)':'Cost_Variance','Planned Cost ($ M)':'PlannedCost','Updated Date':'Updated_Date','Business Case ID':'Business_Case_ID'})  

In [120]:
Projects=Projects.dropna()

In [121]:
Projects.to_csv(directory+'Health_Projects_Out.csv', index=False)

#Cypher code for Neo4j
LOAD CSV WITH HEADERS FROM 'file:///Health_Projects_Out.csv' AS row
MERGE (m:Agency {name:row.Agency_Name,Code:row.Agency_Code})
MERGE (n:Investment {name:row.Investment_Title,Identifier:row.Unique_Investment_Identifier,Business_Case:row.Business_Case_ID})
MERGE (o:Project {name:row.Project_Name,Id:row.Project_ID,Cost_Variance:toFloat(row.Cost_Variance),Planned_Cost:toFloat(row.PlannedCost),Updated_Date:toInteger(row.Updated_Date),Planned_Duration:toInteger(row.Planned_Duration),Project_delay:toInteger(row.Project_delay),Start_days_after_2000:toInteger(row.Start_days_after_2000)})
MERGE (m)-[:invests]->(n)
MERGE (n)-[:pays_for]->(o)

# Pull in strategic investment information

In [42]:
Strategy=pd.read_csv(directory+'Exhibit300A.csv', encoding= 'unicode_escape')

In [43]:
Strategy=Strategy.drop(['Investment Title (Exhibit 53)','Investment Title (Exhibit 300)','Date of Last Change to Activities','Date of Last Update to Activities','Date of Last Update to Activities','Date of Last Update to Activities','Date of Last Change to Contracts','Date of Last Change to Performance Metrics','Date of Last Tech Stat','Budget Year','Date of Last Investment Detail Update','Investment Auto Submission Date'],axis=1)

In [44]:
Strategy=Strategy.drop(['Data Freshness','Date of Last Change to CIO Evaluation','Date of Last Update to CIO Evaluation'],axis=1)

In [45]:
Strategy=Strategy.drop(['IPT Charter Date','Date Investment First Submitted'],axis=1)

In [46]:
Strategy=Strategy.drop(['Business Case ID'],axis=1)

In [47]:
Strategy=Strategy.drop(['CIO Evaluation Color'],axis=1)

In [48]:
Strategy['Date of Last Baseline']=Strategy['Date of Last Baseline'].astype('datetime64')

In [38]:
#Strategy=Strategy.dropna()

In [49]:
Strategy=Strategy.rename(columns={'Agency Name':'Agency_Name',"Bureau Name": "Bureau_Name",'Bureau Code':'Bureau_Code','Number of changes to Baseline':'Number_of_changes_to_Baseline','Evaluation (by Agency CIO)':'Evalu)ation_by_CIO"}'})

In [50]:
Strategy=Strategy.rename(columns={'Evalu)ation_by_CIO"}':'Evaluation_by_CIO','Date of Last Baseline':'Date_Last_Baseline'})

In [54]:
Constraint=Strategy['Agency Code']==9

In [56]:
Strategy=Strategy[Constraint]

## Work out Data Structure as you go. 
## This results in the following Cypher code to be placed into Neo4j
LOAD CSV WITH HEADERS FROM 'file:///Health_Strategy_Out.csv' AS row 
MERGE (m:Agency {Code:row.Agency_Code})
MERGE (o:Bureau {Code:row.Bureau_Code,Name:row.Bureau_Name})
MERGE (n:Investment {Identifier:row.Unique_Investment_Identifier})
CREATE (m)-[:owns]->(o)
CREATE (o)-[:responsible_for]->(n)
SET n.Summary=row.Brief_Summary
SET n.Performance_Gap=row.Summary_of_Performance_Gap
SET n.Evaluation_by_CIO=row.Evaluation_by_CIO
SET n.Changes_to_Baseline=row.Number_of_changes_to_Baseline
SET n.Last_Baseline=row.Date_Last_Baseline

LOAD CSV WITH HEADERS FROM 'file:///Health_Strategy_Out.csv' AS row 
MATCH (m:Agency {Code:row.Agency_Code})
SET m.name = row.Agency_Name
RETURN m

"MERGE matches on the entire pattern you specify within a single clause...The solution is to MERGE on the unique property and then use SET to update additional properties.

In [57]:
Strategy=Strategy.rename(columns={'Unique Investment Identifier':'Unique_Investment_Identifier','Brief Summary':'Brief_Summary',
'Summary of Performance Gap':'Summary_of_Performance_Gap','Agency Code':'Agency_Code'})

In [58]:
Strategy.to_csv(directory+'Health_Strategy_Out.csv', index=False)

# Bring in Contracts file

In [59]:
Contracts=pd.read_csv(directory+'Contracts.csv', encoding= 'unicode_escape')

In [61]:
Constraint=Contracts['Agency Code']==9

In [62]:
Contracts=Contracts[Constraint]

In [63]:
Contracts=Contracts.drop(['Business Case ID','Agency Code','Agency Name','Investment Title','Agency Contract ID','Contract Status','Contracting Agency ID','Contract Number (PIID)','Performance Based Contract (USAspending)','Contract Start Date (USAspending)','Contract End Date (USAspending)','Contract Compete (USAspending)','Base Contract ID (USAspending)'
        ,'Identifying Agency ID (USAspending)','Transaction Number (USAspending)','Timestamp (Base Contract)'],axis=1)

In [64]:
Contracts=Contracts.drop(['IDV Agency ID','Match found in USAspending','Solicitation ID (USAspending)'],axis=1)

In [65]:
Contracts=Contracts.rename(columns={'Unique Investment Identifier':'Unique_Investment_Identifier','Contract ID':'Contract_ID','IDV PIID':'IDV_PIID','Vendor Name (USAspending)':'Vendor_name','Action Obligation Amount (In $ million) (USAspending)':'Contract_size','Contract Description (USAspending)':'Contract_Description'})

## Work out Data Structure as you go. 
## This results in the following Cypher code to be placed into Neo4j

In [122]:
Contracts=Contracts.dropna()

In [123]:
Contracts.to_csv(directory+'Health_Contracts_Out.csv', index=False)

In [ ]:
LOAD CSV WITH HEADERS FROM 'file:///Health_Contracts_Out.csv' AS row 
MERGE (m:Contract {ID:row.Contract_ID})
MERGE (n:Investment {Identifier:row.Unique_Investment_Identifier})
MERGE (o:Supplier {ID:row.IDV_PIID})
SET m.Description=row.Contract_Description
SET m.Contract_size=row.Contract_size
SET o.name=row.Vendor_name
CREATE (n)-[:lets_contract]->(m)
CREATE (n)-[:uses_Supplier]->(o)
CREATE (o)-[:delivers]->(m)


# Bring in Metrics file

In [68]:
Metrics=pd.read_csv(directory+'Performance_Metrics.csv', encoding= 'unicode_escape')

In [69]:
Metrics=Metrics.drop(['Business Case ID','Agency Name','Agency Performance Metric ID','Comment','Updated Date','Updated Time','Reporting Frequency'],axis=1)

In [70]:
Metrics=Metrics.drop(['Target for PY','Actual for PY','Most Recent Actual Results','Measurement Condition'],axis=1)

In [71]:
Metrics=Metrics.rename(columns={'Unique Investment Identifier':'Unique_Investment_Identifier','Performance Metric ID':'Performance_Metric_ID','Metric Description':'Metric_Description','Unit of Measure':'Unit_of_Measure','FEA Performance Measurement Category Mapping':'Measurement_category','Actuals have Met/Not Met Target':'Metric_results'})

In [ ]:
#Metrics=Metrics.dropna()

In [72]:
Constraint=Metrics['Agency Code']==9

In [73]:
Metrics=Metrics[Constraint]

In [75]:
Metrics.to_csv(directory+'Health_Metrics_Out.csv', index=False)

from this, we know that Metric ID is unique
constraint=(Metrics.duplicated(subset='Performance_Metric_ID', keep=False))==True
Metrics[constraint]

## Work out Data Structure as you go. 
## This results in the following Cypher code to be placed into Neo4j
LOAD CSV WITH HEADERS FROM 'file:///Health_Metrics_Out.csv' AS row 
MERGE (m:Metric {ID:row.Performance_Metric_ID})
MERGE (n:Investment {Identifier:row.Unique_Investment_Identifier})
SET m.Description=row.Metric_Description
SET m.Measurement_category=row.Measurement_category
SET m.Metric_results=row.Metric_results
MERGE (n)-[:judged_by]->(m)

# Bring in Business Mapping

In [76]:
Mapping=pd.read_csv(directory+'Exhibit53.csv', encoding= 'unicode_escape')

In [77]:
Constraint=Mapping['Agency Code']==9

In [79]:
Mapping=Mapping[Constraint]

In [80]:
Mapping=Mapping.drop(['Agency Code','Agency Name','Previous UPI','Investment Category','Bureau Code','Bureau Name','Part of Exhibit 53'],axis=1)

In [81]:
Mapping=Mapping.drop(['Mission Delivery And Management Support Area','Line Item Descriptor','Investment Title','Investment Description','Updated Date'],axis=1)

In [82]:
Mapping=Mapping.drop(['Budget Year','XML Request ID','Updated Time','O&M BY Contributions ($ M)','O&M BY Agency Funding ($ M)','O&M CY Contributions ($ M)'],axis=1)

In [83]:
Mapping=Mapping.drop(['O&M CY Agency Funding ($ M)','O&M PY Contributions ($ M)','O&M PY Agency Funding ($ M)','DME BY Contributions ($ M)'],axis=1)

In [84]:
Mapping=Mapping.drop(['Segment Architecture - Agency Segment','Total IT Spending FY2011 (PY) ($ M)','Total IT Spending FY2012 (CY) ($ M)'],axis=1)

In [85]:
Mapping=Mapping.drop(['DME PY Contributions ($ M)','DME CY Contributions ($ M)','DME BY Agency Funding ($ M)'],axis=1)

In [86]:
Mapping=Mapping.rename(columns={'Unique Investment Identifier':'Unique_Investment_Identifier','Type of Investment':'Investment_type'})

In [87]:
Mapping=Mapping.rename(columns={'FEA BRM Mapping - Sub-Function':'Sub-Function','FEA BRM Mapping - Primary Function':'Function','FEA BRM Mapping - Business Area':'Business_Area'})

In [88]:
Mapping=Mapping.rename(columns={'Service Code Mapping - Component':'Service_Component','Service Code Mapping - Primary Function':'Service','Function':'Business Function'})

In [89]:
Mapping=Mapping.rename(columns={'Service Code Mapping - Business Area':'Service Area','Segment Architecture - Federal Standard Segment':'Architecture_code'})

In [90]:
Mapping=Mapping.drop(['Total IT Spending FY2013 (BY) ($ M)','DME PY Agency Funding ($ M)'],axis=1)

In [91]:
Mapping=Mapping.rename(columns={'DME CY Agency Funding ($ M)':'Enhancement_spend_$m'})

In [92]:
Mapping=Mapping.rename(columns={'Enhancement_spend_$M)':'Enhancement_spend_$m'})

In [93]:
Mapping=Mapping.rename(columns={'Service Area':'Service_Area'})

In [94]:
Mapping=Mapping.rename(columns={'Business Function':'Business_Function'})

In [ ]:
#Mapping=Mapping.dropna()

In [95]:
Mapping.to_csv(directory+'Health_Mapping_Out.csv', index=False)

## Code for Neo4j
LOAD CSV WITH HEADERS FROM 'file:///Health_Mapping_Out.csv' AS row 
MERGE (n:Investment {Identifier:row.Unique_Investment_Identifier})
SET n.Enhancement_spend_$m=toFloat(row.Enhancement_spend_$m)
MERGE (o:Service {name:row.Service})
MERGE (p:Business_Area {name:row.Business_Area})
MERGE (q:Business_Function {name:row.Business_Function})
SET n.Investment_type=row.Investment_type
MERGE (n)-[:has_business_function]->(q)
MERGE (q)-[:serves]->(p)
MERGE (n)-[:is_a_type_of]->(o)

# Bring in activities

In [96]:
Activities=pd.read_csv(directory+'Activities.csv', encoding= 'unicode_escape')

In [97]:
Activities=Activities.drop(['Agency Name','Business Case ID','Investment Title','Date of Last Change','Baseline ID'],axis=1)

In [98]:
Activities=Activities.drop(['Cost Variance','Cost Variance Percent'],axis=1)

In [175]:
# Cost Variance and Cost % are unreliable: they give different signs, so dropped

In [99]:
Activities=Activities.drop(['Structure ID','Schedule Variance (in days)','Schedule Variance Percent','Schedule Duration (in days)'],axis=1)

In [100]:
Constraint=Activities['Agency Code']==9

In [101]:
Activities=Activities[Constraint]

In [102]:
Activities=Activities.dropna(subset=['Unique Investment Identifier','Agency Code','Agency Activity ID','Project ID','Activity Name','Start Date Planned','Completion Date Planned',
                                     'Total Costs Planned','Activity Status','Has No Child Activity','Date of Last Update','Unique Activity ID'])

In [103]:
Activities=Activities.drop(['Agency Activity ID'],axis=1)

In [104]:
Activities['Start Date Planned'] = pd.to_datetime(Activities['Start Date Planned'])
Activities['Start Date Projected'] = pd.to_datetime(Activities['Start Date Projected'])
Activities['Start Date Actual'] = pd.to_datetime(Activities['Start Date Actual'])
Activities['Completion Date Planned'] = pd.to_datetime(Activities['Completion Date Planned'])
Activities['Completion Date Projected'] = pd.to_datetime(Activities['Completion Date Projected'])
Activities['Completion Date Actual'] = pd.to_datetime(Activities['Completion Date Actual'])

In [105]:
Activities['Planned_Duration']=np.subtract(Activities['Completion Date Planned'],Activities['Start Date Planned'])

In [106]:
Activities=Activities.drop('Completion Date Planned',axis=1)

In [107]:
Activities['Actual_duration']=np.subtract(Activities['Completion Date Actual'],Activities['Start Date Actual'])
Activities['Actual_start_delay']=np.subtract(Activities['Start Date Actual'],Activities['Start Date Planned'])

In [ ]:
Activities=Activities.drop(['Start Date Actual','Completion Date Actual'],axis=1)

In [111]:
Activities=Activities.rename(columns={'Unique Investment Identifier':'Unique_Investment_Identifier','Project ID':'Project_ID','Activity Name':'Activity_Name','Activity Description':'Activity Description','Key Deliverable / Usable Functionality':'Output_type','Start Date Planned':'Start Date Planned','Total Costs Planned':'Planned_Costs','Activity Status':'Status','Has No Child Activity':'No_Child_Activity','Unique Activity ID':'ID'})

In [ ]:
Activities=Activities.drop(['Unique_Investment_Identifier','Agency Code','Start Date Projected','Start Date Projected'],axis=1)

In [117]:
Activities=Activities.rename(columns={'Activity Description':'Description'})

In [116]:
Activities=Activities.rename(columns={'Start Date Planned':'Start_Date_Planned'})

## Code for Neo4j
LOAD CSV WITH HEADERS FROM 'file:///Health_Activities_Out.csv' AS row 
MERGE (n:Project {Id:row.Project_ID})
MERGE (o:Activity {Id:row.ID})
SET o.name=row.Activity_Name
SET o.description=row.Description
SET o.output_type=row.Output_type
SET o.planned_Costs=row.Planned_Costs
SET o.status=row.Status
SET o.No_Child_Activity=row.No_Child_Activity
SET o.Planned_Duration=row.Planned_Duration
SET o.Actual_duration=row.Actual_duration
SET o.Actual_start_delay=row.Actual_start_delay
SET o.Start_Date_Planned=row.Start_Date_Planned
MERGE (n)-[:has_task]->(o)

# This is the additional code to take subgraph, as a query, to export Cypher code, say for populating a Sandbox
CALL apoc.export.cypher.query(
"MATCH (n:Bureau)-[a]-(o:Investment)-[b]-(p:Project)-[c]-(q:Activity),(o)-[d]-(r:Service),(o)-[e]-(s:Business_Function)-[f]-(t:Business_Area),(o)-[g]-(u:Metric),(o)-[h]-(v:Agency) WHERE n.Name='Small Business Administration' OR  RETURN *","export.cypher",{});

In [119]:
Activities.to_csv(directory+'Health_Activities_Out.csv', index=False)